# Provisioning

## Instances
Instances are sets of vms of various types that can be provisioned across the GreenLake for private cloud stack and a wide range of services like MySQL, Redis, ElasticSearch, PostgreSQL, Tomcat, nginx, Confluence, Jenkins, and more. There are a few important concept differentiators between what we call an instance and what amazon calls an instance. In GreenLake an instance can represent many vms that are of a set. For example. If you wanted to spin up a Mongo sharded replicaset, that requires 7 virtual machines. In GreenLake for private cloud we represent this as a singular instance with a specified layout and then represent all the associated services running within that virtual machine. The serverId represents the compute resource it was provisioned onto, (i.e. the virtual machine).


# Provisioning concepts
HPE GreenLake for private cloud enables provisioning capabilities ranging from individual virtual machines to multi-tier application topologies.

HPE GreenLake for private cloud provides provisioning using the following:

* Virtual images

* Instance types

* Blueprints

Using virtual images, instance types, and blueprints, you can provision:

* Instances

* Apps

## Virtual images
The Virtual Image section of the HPE GreenLake for private cloud dashboard displays a list of synced images from cloud integration. A virtual image is simply a template in VMware vCenter that has been prepared with the right configuration before uploading.

## Instance types
An instance type is derived from a virtual image and can contain variants of the same service. In the following figure, the same Apache service instance type can be used to provision an instance with different versions running on different underlying images.

![Instances Types](Pictures/glc_und_std_lib.png) 


## Blueprints
A blueprint, or application blueprint, allows a user to define an application structure for easy reproducibility and deployment into various environments. Blueprints can be used to mix and match instance types to provision an application dependent on multiple layers, or tiers, of services.

![Instances Types](Pictures/glc-blueprint.png)

## Instances
An instance is a representation of a resource or service. This service may involve several virtual machines. For example, when you create an instance, a particular option may contain several instance configurations. One of these configurations could be a cluster consisting of seven virtual machines. Rather than representing these directly as seven individual instances, HPE GreenLake for private cloud groups them together into a singular instance of a service that contains multiple virtual machines. This model allows for instance actions that can be performed to expand capacity on an instance (either horizontally or vertically). You can apply this concept to something such as a simple Apache web server in which 10 copies of a web server horizontally are scaled out to handle traffic.

When viewing an instance detail page, you can look at details and statistics specific to a virtual machine. This feature helps simplify the management model for tracking these services.

## Apps
An app is a collection of instances linked together using application tiers. Tiers enable you to define segregated sections of connectivity between the various elements or instances within an application. Linking instances together in an application may affect instance environments and provide service discovery capabilities for them to cross-connect.

Once these instances are all linked together in an application concept, this may affect instance environments and provide service discovery capabilities for them to cross-connect.

# Important Note
The HPE GreenLake Central API Acess token has a time to live duration of 15 minutes fro security reasons.

if during the course of the lab, you run an API call and get a result indicating your token is not valid : 


{
  "message": "Unable to read user authentication information.",
  "details": "Invalid access token provided: 'Unauthorized access: Token is not valid'",
  "recommendedActions": [
    "Obtain a valid token and resubmit the request."
  ],
  "errorCode": "UNAUTHORIZED"
}


Please go back to step4 to retrieve the GreenLake Central API Acess token to copy / paste it in the token.txt file.

Do not append content but replace existing token with the new one.

Please re run the next cell each time you update the token in the token.txt file

In [ ]:
##Generating GreenLake Service Client API Token
#the following token is only valid for 15 minutes , please re run the cell to renew the token

ACCESS_TOKEN=$(curl -X POST "https://iam.us1.greenlake-hpe.com/oauth2/buu0lcj6alfdq0ajli4g/ausanc3firBcvcYdT357/v1/token" --header 'Content-Type: application/x-www-form-urlencoded' \
--data-urlencode 'client_id={{ STUDENT_CLIENTID }}' \
--data-urlencode 'client_secret={{ STUDENT_CLIENTSECRET }}' \
--data-urlencode 'grant_type=client_credentials' \
--data-urlencode 'scope=hpe-tenant' |jq -r  .access_token) 

echo "your api token is: $ACCESS_TOKEN"

# VMaaS Variables
API_URL="https://client.greenlake.hpe.com/api/iac-vmaas/v1"
space="HPEDEV%20Workshop-on-Demand%20workshops"
location="HPE"

##GreenLake Central Login Details
echo "your operation context is:" "student{{ STDID }}"  "with Password:" "{{ PASSSTU }}{{ PASSSTU }}"

Let's start by checking how many instances are running here

In [ ]:
echo "Listing all available instances now"
curl -s -X GET \
  "$API_URL/instances?space=$space&location=$location" \
  -H "Authorization: $ACCESS_TOKEN" |jq -r

We will need the student group id as well, let 's get it as a variable now.


In [ ]:
echo "Getting your group Id now"

groupid=$(curl -s "$API_URL/groups?space=$space&location=$location&name=student{{ STDID }}" -H "Authorization: $ACCESS_TOKEN" |jq -r '.groups[].id')
echo "your student groupid is: $groupid"

## Create now your first instance!

You will deploy here a centos 7 VM named after your studentname ("name": "api-vm-student{{ STDID }})

### Creating an instance
- POST /api/instances

Payload: image  here


### Just before we start...

We will take a quick look at the existing tasks available for us. There are many ways to create a user at the deployment time. It could part of the overall student settings or you can actually use a task that will be processed at the end of the deployment as part of a workflow.

The next cell will list all available tasks in the current setup. You will notice that there is a simple bash task named create user. We will leverage it to add a student user to the instance we will create.

You can also take a look at the create instance API call and see at the very end of the call the task associated.

In [ ]:
echo "Listing existing tasks "

curl -s "$API_URL/tasks?space=$space&location=$location" \
  -H "Authorization: $ACCESS_TOKEN" | jq -r

Look for 
"id": 22,
"accountId": 2,
"name": "Create user",

In [ ]:
echo "Creating your first HPE GreenLake Private Cloud Instance"

curl -s --request POST "$API_URL/instances?space=$space&location=$location" \
--header "Authorization: Bearer $ACCESS_TOKEN" \
--header 'Content-Type: application/json' \
--data-raw "{ "zoneId": 1, "instance": { "name": "vm-student{{ STDID }}-api", "site": { "id": $groupid }, "instanceType": { "code": "glhc-vanilla-centos" }, "layout": { "id": 1159 }, "plan": { "id": 402 } }, "volumes": [ { "name": "root", "size": 15, "datastoreId": "auto" } ], "networkInterfaces": [ { "network": { "id": 7 }, "networkInterfaceTypeId": 4 } ], "config": { "resourcePoolId": 2, "vmwareFolderId": "group-v41" }, "tags": [ {"name": "api-provisioned", "value": "True"} ], "workflow": {"taskSetId": 22}  }"

Let's now get the the VM id to be used later for managing it. We'll make it a variable for ease of use.

In [ ]:
echo "Getting your VM Id now"

student{{ STDID }}vmid=$(curl -s --request GET "$API_URL/instances?space=$space&location=$location&max=25&name=vm-student{{ STDID }}-api" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instances[0].id')


echo "your vm id is "$student{{ STDID }}vmid

### Note: The provisionning on the instance is quite quick. It is a matter of a minute ! We can check the its status with the following api call

In [ ]:
echo "Checking provisionning status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

Once created and as for any Virtual machine, many operations are available to you:
- Update
- Stop
- Start
- Restart
- Delete
- Backup
- Clone
- Resize

Let's see these in details now.

### Updating an instance

[Updates an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/UpdatesAnInstance/)

- PUT /instances/:id

**Note: Site is alias of Group**

- --name VALUE Name
- --description VALUE Description
- --environment VALUE Environment
- --group GROUP Group Name or ID
- --labels [LIST] Labels (keywords) in the format 'foo, bar'
- --tags LIST Tags in the format 'name:value, name:value'. This will add and remove tags.
- --add-tags TAGS Add Tags in the format 'name:value, name:value'. This will only add/update tags.
- --remove-tags TAGS Remove Tags in the format 'name, name:value'. This removes tags, the :value component is optional and must match if passed.
- --power-schedule-type ID Power Schedule Type ID
- --owner USER Owner Username or ID



Here, we will simply update the description field:
It should be set to null as we did not specify it at the instance creation time

In [ ]:
echo "Checking the description field value"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.description'

Let's Modify it now

In [ ]:
echo "Updating now the description field value"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN " \
-H 'Content-Type: application/json' \
--data-raw '{ "instance": {
  "description": "Description updated by API call by student{{ STDID }}"
  }}' | jq -r

### Let's check the difference now : Description field should no longer be null

In [ ]:
echo "Checking the updated description field value"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.description'

### Let's check again the status of the instance now, it should be running by now.

In [ ]:
echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### Stop an instance

[Stops an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/StopsAnInstance/)

- PUT /instances/:id/stop?server=true&muteMonitoring=false

- --mute-monitoring [on|off] Mute monitoring. Default is off.
- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
echo "Stopping your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/stop?space=$space&location=$location&server=true&muteMonitoring=false" \
-H "Authorization: $ACCESS_TOKEN" | jq -r

echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### Start an instance

[Starts an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/StartsAnInstance/)

- PUT /instances/:id/start?server=true

- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
echo "Starting your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/start?space=$space&location=$location&server=true&muteMonitoring=true" \
-H "Authorization: $ACCESS_TOKEN" | jq -r

echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### Restart an instance

[Restarts an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/RestartsInstance/)

- PUT /instances/:id/restart?server=true&muteMonitoring=true

- --mute-monitoring [on|off] Mute monitoring. Default is on.
- -y, --yes Auto Confirm
- -q, --quiet No Output, do not print to stdout

In [ ]:
echo "Restarting your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/restart?space=$space&location=$location&server=true&muteMonitoring=true" \
-H "Authorization: $ACCESS_TOKEN" | jq -r


echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### Suspend an instance

[Suspends an Instance GreenLake Doc link ](https://developer.greenlake.hpe.com/docs/greenlake/services/private-cloud/public/openapi/private-cloud-cmp-latest/operation/SuspendsAnInstance/)

- PUT /instances/:id/suspend?

In [ ]:
echo "Suspending your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/suspend?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r

echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### Start again your instance

In [ ]:
echo "Starting again your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request PUT "$API_URL/instances/$student{{ STDID }}vmid/start?space=$space&location=$location&server=true&muteMonitoring=true" \
-H "Authorization: $ACCESS_TOKEN" | jq -r


echo "Checking status of your student{{ STDID }} instance with id $student{{ STDID }}vmid"

curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" | jq -r '.instance.status'

### What about accessing the vm now ? 
Let's get the details for our centos instance. An IP address might help to connect. Some crededntials too.

In [ ]:
echo " Retrieving your student{{ STDID }} instance IP Address now"

student{{ STDID }}vmip=$(curl -s --request GET "$API_URL/instances/$student{{ STDID }}vmid?space=$space&location=$location" \
-H "Authorization: $ACCESS_TOKEN" |jq -r '.instance.connectionInfo[0].ip')

echo "Your student{{ STDID }} instance IP Address is $student{{ STDID }}vmip"

#### Check if vm is reachable 

In [ ]:
ping -c 5 $student{{ STDID }}vmip

####  Connect now to the vm

Click on the + button on the right hand side of the notebook UI to reach the  launcher page. Click then on the terminal tile in the other category at the bottom of the page.

This will open up a new tab allowing you a proper cli access to run the ssh commands.

user is `student` and password is `P@ssWkshp`


![SSH](Pictures/wod-greenlakeapi-ssh.PNG)

copy paste the content of the next cell in the terminal window to excute it

In [ ]:
ssh student@$$student{{ STDID }}vmip

## Summary:
In this lab, you created your first instance leveraging the HPE GreenLake VMAAS API. You also managed to stop, start, restart this instance.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#008000;"></i>&nbsp;&nbsp;Next Steps

Lab 3: Advanced Management of your Instance 

</br>
 <a href="1-WKSHP-VMAAS-Authentication.ipynb" target="New" title="Back:Authentication"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-VMAAS-Instance-Advanced.ipynb#sc" target="New" title="Next:Advanced Management of your Instance"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="3-WKSHP-VMAAS-Instance-Advanced.ipynb#sc" target="New" title="Next: Advanced Management of your Instance"><i class="fas fa-chevron-circle-right" style="color:#008000;"></i></a></h2>